In [7]:
import h2o
h2o.init()
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,5 days 20 hours 48 mins
H2O cluster timezone:,America/Santiago
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.3
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_maravenag_3wzuvj
H2O cluster total nodes:,1
H2O cluster free memory:,3.271 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [8]:
data = h2o.upload_file("cereal.csv", destination_frame="train")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
data = data.drop("name")

In [10]:
data.describe()

Rows:77
Cols:15




,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
type,enum,enum,int,int,int,int,real,real,int,int,int,int,real,real,real
mins,,,50.0,1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,1.0,0.5,0.25,18.042851
mean,,,106.8831168831169,2.545454545454545,1.0129870129870129,159.67532467532473,2.1519480519480525,14.597402597402601,6.922077922077923,96.07792207792208,28.246753246753244,2.2077922077922074,1.0296103896103896,0.8210389610389608,42.665704987012965
maxs,,,160.0,6.0,5.0,320.0,14.0,23.0,15.0,330.0,100.0,3.0,1.5,1.5,93.704912
sigma,,,19.484119056820845,1.094789748445534,1.0064725594803927,83.83229524009316,2.3833639643872226,4.278956280325907,4.4448853924193585,71.2868125092621,22.342522500566307,0.832524100135788,0.15047679973689213,0.23271613844691388,14.047288743735217
zeros,,,0,0,27,9,19,0,7,0,8,0,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,N,C,70.0,4.0,1.0,130.0,10.0,5.0,6.0,280.0,25.0,3.0,1.0,0.33,68.402973
1,Q,C,120.0,3.0,5.0,15.0,2.0,8.0,8.0,135.0,0.0,3.0,1.0,1.0,33.983679
2,K,C,70.0,4.0,1.0,260.0,9.0,7.0,5.0,320.0,25.0,3.0,1.0,0.33,59.425505


In [11]:
data['mfr'] = data['mfr'].asfactor()
data['type'] = data['type'].asfactor()

In [12]:
data.head()

mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
N,C,70,4,1,130,10,5,6,280,25,3,1,0.33,68.403
Q,C,120,3,5,15,2,8,8,135,0,3,1,1,33.9837
K,C,70,4,1,260,9,7,5,320,25,3,1,0.33,59.4255
K,C,50,4,0,140,14,8,0,330,25,3,1,0.5,93.7049
R,C,110,2,2,200,1,14,8,-1,25,3,1,0.75,34.3848
G,C,110,2,2,180,1.5,10.5,10,70,25,1,1,0.75,29.5095
K,C,110,2,0,125,1,11,14,30,25,2,1,1,33.1741
G,C,130,3,2,210,2,18,8,100,25,3,1.33,0.75,37.0386
R,C,90,2,1,200,4,15,6,125,25,1,1,0.67,49.1203
P,C,90,3,0,210,5,13,5,190,25,3,1,0.67,53.3138


In [13]:
target = "rating"

In [14]:
train = data

In [15]:
train_cols = [x for x in train.col_names if x not in ['rating']]

In [17]:
### using autoencoder for the data

In [18]:
from h2o.estimators import H2OAutoEncoderEstimator

autoencoder = H2OAutoEncoderEstimator(activation="Tanh",
                              hidden=[25],
                              ignore_const_cols=False,
                              epochs=2000)

autoencoder.train(x=train_cols, training_frame=data)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [19]:
autoencoder._model_json["output"]["training_metrics"]


ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 0.00021819916412538058
RMSE: 0.014771566068815472


In [20]:
train = data

In [21]:
train_deep_feats = autoencoder.predict(train)
train_deep_feats = train_deep_feats.cbind(train['rating'])

deep_cols = [x for x in train_deep_feats.col_names if x not in ['rating']]

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [22]:
train_deep_feats.head()

reconstr_mfr.A,reconstr_mfr.G,reconstr_mfr.K,reconstr_mfr.N,reconstr_mfr.P,reconstr_mfr.Q,reconstr_mfr.R,reconstr_mfr.missing(NA),reconstr_type.C,reconstr_type.H,reconstr_type.missing(NA),reconstr_calories,reconstr_protein,reconstr_fat,reconstr_sodium,reconstr_fiber,reconstr_carbo,reconstr_sugars,reconstr_potass,reconstr_vitamins,reconstr_shelf,reconstr_weight,reconstr_cups,rating
-0.00497518,0.00311861,-0.00200944,0.998809,0.000871987,-0.0056628,0.00549959,0.000718734,0.999898,0.00469437,-3.36617e-06,69.9097,3.98599,1.06699,130.467,10.0114,4.99246,5.97063,282.369,24.8075,2.99971,1.00265,0.330342,68.403
-0.00210131,0.00510326,-0.00393941,3.79452e-05,0.00293618,0.997899,0.00351094,0.000128251,0.999961,0.00496302,-3.51908e-05,120.235,3.00038,4.90159,14.9967,2.01315,8.00843,7.96968,135.941,-0.203811,2.99807,1.0031,0.998671,33.9837
-0.0030804,0.00117463,0.999797,-0.00224186,-0.00107014,-0.00550317,-0.00264055,0.00081694,0.99999,-0.00216336,-5.01668e-05,70.1627,4.02397,0.964114,259.803,9.13272,6.99415,4.99585,316.615,24.656,2.99407,1.00243,0.332678,59.4255
0.00951971,-0.0013626,0.999791,0.018725,0.00353472,0.0174875,0.0110954,0.00233181,0.999994,-0.00434593,0.00160106,51.4392,3.89587,0.0187613,138.942,13.1123,7.97928,0.139951,327.827,24.5963,3.00427,0.994691,0.486307,93.7049
-0.00308796,-1.2433e-05,-0.00435196,-0.0133778,-0.00594969,0.0125269,0.998037,-0.000128077,0.99999,-0.00465225,0.00149461,110.07,1.96617,2.02511,195.467,0.958447,14.1117,7.93642,-2.92344,24.3854,2.99905,0.996452,0.771298,34.3848
-0.0158275,0.999437,-0.00612284,-0.00991643,-0.000334404,0.01519,0.0159494,-0.00078266,0.999995,0.00946054,0.000740783,110.286,2.02147,2.1181,177.179,1.43181,10.3592,9.92023,70.9617,23.2755,1.00975,1.01422,0.74914,29.5095
0.00265424,0.00716189,0.999803,0.00943574,0.00490115,-0.0100473,0.0016962,0.00124943,0.999997,-0.00497173,0.000165166,111.227,1.96239,0.0222576,125.06,0.7638,11.1524,13.9897,29.7961,25.7943,2.01504,1.00826,0.990922,33.1741
-0.00639129,0.999489,-0.0138927,0.00487907,0.00100911,-0.00200827,0.00379103,-0.000603487,0.999997,0.0124217,0.00123721,131.328,2.99061,2.02186,214.209,2.02308,18.2274,7.81467,97.0209,24.5649,3.00329,1.3302,0.755329,37.0386
-0.00243767,0.00720204,-0.00530003,0.0045142,0.00555463,-0.0027644,0.998108,0.000879148,0.999966,0.00963978,0.000143329,90.1656,1.9853,1.01991,201.444,4.03015,15.071,5.95534,125.56,24.7659,1.01269,0.993367,0.668377,49.1203
-0.0488307,0.0204212,-0.00730144,0.017686,0.998735,-0.0229389,0.0334541,-0.000403359,0.999961,-0.00624784,-0.000685631,89.5214,2.95138,0.0296167,203.186,4.86451,12.6987,5.17153,192.331,24.6472,2.97099,1.00953,0.668167,53.3138
